## 1. Installation & Configuration

In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from newsapi.newsapi_client import NewsApiClient
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ Bibliothèques chargées")

d:\Documents\ECE\2025\PFE\FinsightAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Bibliothèques chargées


In [2]:
import os
from dotenv import load_dotenv
from pathlib import Path
load_dotenv(Path(".env"))
newsapi = NewsApiClient(api_key=os.getenv("NEWSAPI_API_KEY"))

# Device (CPU ou GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


## 2. Chargement de votre modèle FinBERT

In [3]:
# Charger votre modèle fine-tuné
MODEL_PATH = "./news_finbert_sentiment_model"

print("Chargement du modèle FinBERT...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=2,
    use_safetensors=True
)
model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
print("✓ Modèle FinBERT chargé et prêt")

Chargement du modèle FinBERT...
✓ Modèle FinBERT chargé et prêt


## 3.1. Configuration des actifs et mots-clés

### Configuration des actifs avec leurs mots-clés spécifiques

In [4]:
ASSETS_CONFIG = {
    # Indices
    'SP500': {
        'type': 'Index',
        'keywords': ['S&P 500', 'SP500', 'Wall Street', 'Dow Jones', 'Nasdaq', 'Trump', 'Fed', 'Federal Reserve', 'US economy', 'American stocks']
    },
    'CAC40': {
        'type': 'Index',
        'keywords': ['CAC 40', 'CAC40', 'Paris Bourse', 'Euronext Paris', 'Macron', 'French economy', 'France stocks']
    },
    'GER30': {
        'type': 'Index',
        'keywords': ['DAX', 'DAX 30', 'GER30', 'Frankfurt', 'German economy', 'Scholz', 'Germany stocks']
    },
    
    # Entreprises Tech
    'AAPL': {
        'type': 'Company',
        'keywords': ['Apple', 'AAPL', 'iPhone', 'iPad', 'Mac', 'Tim Cook', 'iOS', 'App Store']
    },
    'AMZN': {
        'type': 'Company',
        'keywords': ['Amazon', 'AMZN', 'AWS', 'Bezos', 'Prime', 'e-commerce', 'cloud computing']
    },
    'TSLA': {
        'type': 'Company',
        'keywords': ['Tesla', 'TSLA', 'Elon Musk', 'electric vehicle', 'EV', 'Model 3', 'Model Y', 'Cybertruck']
    },
    
    # Entreprises Françaises
    'SAN': {
        'type': 'Company',
        'keywords': ['Sanofi', 'SAN', 'pharmaceutical', 'vaccine', 'healthcare']
    },
    'HO': {
        'type': 'Company',
        'keywords': ['Thales', 'HO', 'defense', 'aerospace', 'cybersecurity']
    },
    'MC': {
        'type': 'Company',
        'keywords': ['LVMH', 'Louis Vuitton', 'Bernard Arnault', 'luxury', 'Dior', 'Moët Hennessy']
    },
    'ENGI': {
        'type': 'Company',
        'keywords': ['Engie', 'ENGI', 'energy', 'renewable', 'gas', 'electricity']
    },
    'TTE': {
        'type': 'Company',
        'keywords': ['TotalEnergies', 'Total', 'TTE', 'oil', 'gas', 'energy', 'petroleum']
    },
    'AIR': {
        'type': 'Company',
        'keywords': ['Airbus', 'AIR', 'aircraft', 'A320', 'A350', 'aviation', 'aerospace']
    },
    
    # Commodités
    'OIL': {
        'type': 'Commodity',
        'keywords': ['crude oil', 'oil price', 'petroleum', 'Brent', 'WTI', 'OPEC', 'energy prices']
    },
    'GOLD': {
        'type': 'Commodity',
        'keywords': ['gold', 'gold price', 'precious metal', 'bullion', 'gold market']
    }
}

def build_query_from_keywords(keywords):
    """Construit une requête OR pour NewsAPI"""
    return ' OR '.join([f'"{kw}"' for kw in keywords])

print(f"✓ {len(ASSETS_CONFIG)} actifs configurés")

✓ 14 actifs configurés


## 3.2 Fonction de récupération de news

In [5]:
def fetch_financial_news(query=None, 
                         country=None, 
                         language="en",
                         page_size=100,
                         categories=None,
                         days_back=7,
                         use_asset_keywords=False):
    """
    Récupère des news financières via NewsAPI
    
    Args:
        query: Mots-clés personnalisés (ex: "Apple OR Tesla")
        country: Code pays ISO (ex: "us", "fr", "gb")
        language: Code langue ("en", "fr", etc.)
        page_size: Nombre d'articles (max 100)
        categories: Liste de catégories ['business', 'health', 'science', 'technology']
        days_back: Nombre de jours en arrière
        use_asset_keywords: Si True, utilise les mots-clés des actifs configurés
    
    Returns:
        Liste de dictionnaires avec les news + asset mapping
    """
    
    from_date = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')
    
    if categories is None:
        categories = ['business', 'health', 'science', 'technology']
    
    all_news = []
    
    # PARTIE 1: Récupération par CATÉGORIES (top-headlines)
    print("\n" + "="*80)
    print("📰 RÉCUPÉRATION PAR CATÉGORIES (Top Headlines)")
    print("="*80)
    
    for category in categories:
        try:
            print(f"\n🔍 Catégorie: {category}...")
            
            articles = newsapi.get_top_headlines(
                q=query if query and not use_asset_keywords else None,
                language=language,
                page_size=page_size,
                category=category,
                country=country
            )
            
            for article in articles.get('articles', []):
                all_news.append({
                    'title': article.get('title', ''),
                    'description': article.get('description', ''),
                    'content': article.get('content', ''),
                    'source': article.get('source', {}).get('name', 'Unknown'),
                    'url': article.get('url', ''),
                    'published_at': article.get('publishedAt', ''),
                    'author': article.get('author', 'Unknown'),
                    'news_category': category,
                    'query_keyword': 'CATEGORY_' + category.upper(),
                    'asset_ticker': None,  # Sera rempli après
                    'asset_type': None
                })
            
            print(f"   ✓ {len(articles.get('articles', []))} articles")
        
        except Exception as e:
            print(f"   ❌ Erreur: {e}")
            continue
    
    # PARTIE 2: Récupération par MOTS-CLÉS des actifs (everything)
    if use_asset_keywords:
        print("\n" + "="*80)
        print("🎯 RÉCUPÉRATION PAR ACTIFS (Mots-clés spécifiques)")
        print("="*80)
        
        for ticker, config in ASSETS_CONFIG.items():
            try:
                asset_query = build_query_from_keywords(config['keywords'])
                print(f"\n🔍 Actif: {ticker} ({config['type']})...")
                print(f"   Query: {asset_query[:100]}...")
                
                articles = newsapi.get_everything(
                    q=asset_query,
                    language=language,
                    from_param=from_date,
                    page_size=page_size,
                    sort_by='publishedAt'
                )
                
                for article in articles.get('articles', []):
                    all_news.append({
                        'title': article.get('title', ''),
                        'description': article.get('description', ''),
                        'content': article.get('content', ''),
                        'source': article.get('source', {}).get('name', 'Unknown'),
                        'url': article.get('url', ''),
                        'published_at': article.get('publishedAt', ''),
                        'author': article.get('author', 'Unknown'),
                        'news_category': 'KEYWORD_SEARCH',
                        'query_keyword': ticker,
                        'asset_ticker': ticker,
                        'asset_type': config['type']
                    })
                
                print(f"   ✓ {len(articles.get('articles', []))} articles")
            
            except Exception as e:
                print(f"   ❌ Erreur pour {ticker}: {e}")
                continue
    
    # Supprimer les doublons basés sur l'URL
    unique_news = []
    seen_urls = set()
    for news in all_news:
        if news['url'] not in seen_urls:
            seen_urls.add(news['url'])
            unique_news.append(news)
    
    print(f"\n{'='*80}")
    print(f"✅ TOTAL: {len(unique_news)} articles uniques (sur {len(all_news)} récupérés)")
    print(f"{'='*80}\n")
    
    return unique_news

In [6]:
# Ajouter après la cellule fetch_financial_news

## 3.2. Fonction de matching automatique des actifs

def match_news_to_assets(news_text):
    """
    Détecte automatiquement les actifs mentionnés dans un texte
    
    Args:
        news_text: Titre + description de la news
    
    Returns:
        Liste de tuples (ticker, type) des actifs trouvés
    """
    text_lower = news_text.lower()
    matched_assets = []
    
    for ticker, config in ASSETS_CONFIG.items():
        for keyword in config['keywords']:
            if keyword.lower() in text_lower:
                matched_assets.append((ticker, config['type']))
                break  # Un seul match par actif suffit
    
    return matched_assets

# Test
test_text = "Apple's iPhone sales surge as Tesla announces new Model Y"
matches = match_news_to_assets(test_text)
print(f"Test: '{test_text}'")
print(f"Actifs détectés: {matches}")

Test: 'Apple's iPhone sales surge as Tesla announces new Model Y'
Actifs détectés: [('AAPL', 'Company'), ('TSLA', 'Company'), ('HO', 'Company')]


In [7]:
# À ajouter après la cellule "3.2 Fonction de matching automatique des actifs"

## 3.3 Filtrage des News Non-Financières

# Catégories et sources à exclure
EXCLUDED_SOURCES = {
    'DogTime', 'Parade', 'AOL.com', 'Yahoo Entertainment',
    'Slate Magazine', 'Gizmodo.com', 'Tom\'s Guide', 'Mashable',
    'Variety', 'The Verge', 'Wired', 'Gaming', 'Kotaku',
    'IGN', 'PC Gamer', 'Polygon'
}

EXCLUDED_KEYWORDS = {
    'gaming', 'video game', 'esports', 'game', 'player',
    'celebrity', 'entertainment', 'movie', 'music', 'actor', 'actress',
    'sports', 'football', 'basketball', 'soccer', 'tennis',
    'health tips', 'diet', 'workout', 'fitness', 'recipe',
    'pet', 'animal', 'dog', 'cat', 'puppy',
    'dating', 'relationship', 'advice', 'love',
    'celebrity gossip', 'reality tv', 'streaming service'
}

def filter_non_financial_news(df):
    """
    Filtre les news non-financières (Niveau 1 & 2)
    
    Args:
        df: DataFrame avec les news
    
    Returns:
        DataFrame filtré (news financières uniquement)
    """
    if df.empty:
        return df
    
    before_count = len(df)
    
    # NIVEAU 1: Exclure par source
    print("\n🔍 FILTRAGE NIVEAU 1 - Sources exclues")
    print(f"   Avant: {len(df)} news")
    
    df_filtered = df[~df['source'].isin(EXCLUDED_SOURCES)].copy()
    removed_by_source = before_count - len(df_filtered)
    print(f"   Après: {len(df_filtered)} news (-{removed_by_source})")
    
    # NIVEAU 2: Exclure par mots-clés
    print("\n🔍 FILTRAGE NIVEAU 2 - Mots-clés non-financiers")
    before_keywords = len(df_filtered)
    
    title_description = (
        df_filtered['title'].str.lower().fillna('') + ' ' + 
        df_filtered['description'].str.lower().fillna('')
    )
    
    # Exclure si un mot-clé non-financier est présent
    mask = ~title_description.str.contains(
        '|'.join(EXCLUDED_KEYWORDS),
        regex=True,
        na=False
    )
    df_filtered = df_filtered[mask].copy()
    removed_by_keywords = before_keywords - len(df_filtered)
    print(f"   Avant: {before_keywords} news")
    print(f"   Après: {len(df_filtered)} news (-{removed_by_keywords})")
    
    print(f"\n✅ TOTAL FILTRÉ: {before_count} → {len(df_filtered)} news")
    print(f"   Éliminées: {before_count - len(df_filtered)} ({(before_count - len(df_filtered)) / before_count * 100:.1f}%)")
    
    return df_filtered


# Test du filtre
print("="*80)
print("TEST DU FILTRE NON-FINANCIER")
print("="*80)

test_articles = pd.DataFrame({
    'title': [
        'Apple Stock Hits Record High',
        'Top 10 Gaming Trends in 2026',
        'Tesla Announces New Factory',
        'Celebrity Gossip: Brangelina Reunion?',
        'Fed Raises Interest Rates to 5.5%',
        'Best Workout Tips for Summer'
    ],
    'description': [
        'AAPL closes at $250',
        'Best video games to play now',
        'Gigafactory expansion plans',
        'Hollywood news update',
        'Federal Reserve monetary policy decision',
        'Fitness advice for beginners'
    ],
    'source': ['Bloomberg', 'IGN', 'Reuters', 'Variety', 'CNBC', 'Fitness Magazine'],
    'sentiment': ['Positive', 'Positive', 'Positive', 'Neutral', 'Negative', 'Positive'],
    'confidence': [0.9, 0.95, 0.88, 0.92, 0.91, 0.87],
    'url': [f'http://example.com/{i}' for i in range(6)]
})

print("\n📰 AVANT FILTRAGE:")
print(test_articles[['title', 'source']])

test_filtered = filter_non_financial_news(test_articles)

print("\n📰 APRÈS FILTRAGE:")
print(test_filtered[['title', 'source']])

TEST DU FILTRE NON-FINANCIER

📰 AVANT FILTRAGE:
                                   title            source
0           Apple Stock Hits Record High         Bloomberg
1           Top 10 Gaming Trends in 2026               IGN
2            Tesla Announces New Factory           Reuters
3  Celebrity Gossip: Brangelina Reunion?           Variety
4      Fed Raises Interest Rates to 5.5%              CNBC
5           Best Workout Tips for Summer  Fitness Magazine

🔍 FILTRAGE NIVEAU 1 - Sources exclues
   Avant: 6 news
   Après: 4 news (-2)

🔍 FILTRAGE NIVEAU 2 - Mots-clés non-financiers
   Avant: 4 news
   Après: 2 news (-2)

✅ TOTAL FILTRÉ: 6 → 2 news
   Éliminées: 4 (66.7%)

📰 APRÈS FILTRAGE:
                               title     source
0       Apple Stock Hits Record High  Bloomberg
4  Fed Raises Interest Rates to 5.5%       CNBC


## 4. Fonction d'analyse de sentiment

In [8]:
def analyze_sentiment(text, model, tokenizer, device, max_length=512):
    """
    Analyse le sentiment d'un texte avec votre modèle FinBERT
    
    Args:
        text: Texte à analyser
        model: Modèle FinBERT
        tokenizer: Tokenizer
        device: CPU ou GPU
        max_length: Longueur max (512 pour news)
    
    Returns:
        dict avec sentiment, confiance, probabilités
    """
    
    if not text or len(text.strip()) == 0:
        return {
            'sentiment': 'Unknown',
            'confidence': 0.0,
            'prob_negative': 0.5,
            'prob_positive': 0.5
        }
    
    # Tokenization
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Prédiction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)[0]
        
        prediction = torch.argmax(logits, dim=1).item()
        confidence = probabilities[prediction].item()
    
    sentiment_label = "Positive" if prediction == 1 else "Negative"
    
    return {
        'sentiment': sentiment_label,
        'confidence': confidence,
        'prob_negative': probabilities[0].item(),
        'prob_positive': probabilities[1].item()
    }

## 5. Pipeline complet : Fetch + Analyze

In [9]:
# Remplacer la cellule "5. Pipeline complet : Fetch + Analyze"

def analyze_news_pipeline(query=None, country=None, page_size=25, categories=None, use_asset_keywords=True, apply_filter=True):
    """
    Pipeline complet : récupère les news et analyse le sentiment
    
    Args:
        query: Mots-clés de recherche personnalisés
        country: Code pays ISO
        page_size: Nombre d'articles par catégorie/actif
        categories: Liste de catégories
        use_asset_keywords: Si True, boucle sur tous les actifs configurés
        apply_filter: Si True, applique le filtre non-financier
    
    Returns:
        DataFrame avec news + analyse + mapping actifs
    """
    
    print("="*80)
    print("🚀 PIPELINE COMPLET - NEWS + SENTIMENT + ASSET MAPPING")
    print("="*80)
    
    # 1. Récupérer les news
    news_list = fetch_financial_news(
        query=query, 
        country=country, 
        page_size=page_size,
        categories=categories,
        use_asset_keywords=use_asset_keywords
    )
    
    if not news_list:
        print("❌ Aucune news récupérée")
        return pd.DataFrame()
    
    # 2. Analyser chaque news + mapper aux actifs
    print("\n🤖 ANALYSE DE SENTIMENT + MAPPING ACTIFS")
    print("="*80)
    
    results = []
    
    for i, news in enumerate(news_list, 1):
        full_text = f"{news['title']} {news['description']}"
        
        # Analyse de sentiment
        sentiment_result = analyze_sentiment(full_text, model, tokenizer, device)
        
        # Si asset_ticker n'est pas déjà défini (cas des catégories), on le détecte
        if news['asset_ticker'] is None:
            matched_assets = match_news_to_assets(full_text)
            
            # Si aucun actif détecté, on crée une seule ligne "Unknown"
            if not matched_assets:
                matched_assets = [('UNKNOWN', 'Unknown')]
            
            # Créer une ligne pour chaque actif détecté
            for ticker, asset_type in matched_assets:
                results.append({
                    'asset_ticker': ticker,
                    'asset_type': asset_type,
                    'query_keyword': news['query_keyword'],
                    'news_category': news['news_category'],
                    'title': news['title'],
                    'description': news.get('description', ''),
                    'source': news['source'],
                    'published_at': news['published_at'],
                    'sentiment': sentiment_result['sentiment'],
                    'confidence': sentiment_result['confidence'],
                    'prob_negative': sentiment_result['prob_negative'],
                    'prob_positive': sentiment_result['prob_positive'],
                    'url': news['url']
                })
        else:
            # Asset déjà défini (cas des mots-clés)
            results.append({
                'asset_ticker': news['asset_ticker'],
                'asset_type': news['asset_type'],
                'query_keyword': news['query_keyword'],
                'news_category': news['news_category'],
                'title': news['title'],
                'description': news.get('description', ''),
                'source': news['source'],
                'published_at': news['published_at'],
                'sentiment': sentiment_result['sentiment'],
                'confidence': sentiment_result['confidence'],
                'prob_negative': sentiment_result['prob_negative'],
                'prob_positive': sentiment_result['prob_positive'],
                'url': news['url']
            })
        
        if i % 25 == 0:
            print(f"   ✓ Analysé {i}/{len(news_list)} news...")
    
    df = pd.DataFrame(results)
    
    # 3. Appliquer le filtre non-financier (NOUVEAU)
    if apply_filter and not df.empty:
        print(f"\n{'='*80}")
        print("🧹 FILTRAGE NON-FINANCIER")
        print(f"{'='*80}")
        df = filter_non_financial_news(df)
    
    print(f"\n{'='*80}")
    print(f"✅ {len(df)} associations news-actifs analysées")
    print(f"{'='*80}\n")
    
    return df

## 6. Exemples d'utilisation

In [10]:
# Remplacer la cellule "6. Exemples d'utilisation"

## 6. Exemples d'utilisation avec filtrage

# EXEMPLE 1: Récupération complète avec FILTRAGE ACTIF
df_finance = analyze_news_pipeline(
    page_size=100,
    categories=['business', 'health', 'science', 'technology'],
    use_asset_keywords=True,
    apply_filter=True  # ✅ Active le filtrage
)

# Aperçu des résultats
print("\n📊 APERÇU DES RÉSULTATS")
print(df_finance[['asset_ticker', 'asset_type', 'query_keyword', 'title', 'sentiment', 'confidence']].head(20))

# Statistiques globales
print("\n📈 STATISTIQUES GLOBALES:")
print(f"Total news-actifs: {len(df_finance)}")
print(f"Actifs uniques: {df_finance['asset_ticker'].nunique()}")
print(f"Sources uniques: {df_finance['source'].nunique()}")

# Statistiques par actif
print("\n📊 STATISTIQUES PAR ACTIF:")
asset_stats = df_finance.groupby('asset_ticker').agg({
    'title': 'count',
    'sentiment': lambda x: (x == 'Positive').sum() / len(x) if len(x) > 0 else 0,
    'confidence': 'mean'
}).rename(columns={
    'title': 'count',
    'sentiment': 'positive_ratio',
    'confidence': 'avg_confidence'
}).sort_values('count', ascending=False)

print(asset_stats.head(15))

# Statistiques par type d'actif
print("\n📊 STATISTIQUES PAR TYPE:")
type_stats = df_finance.groupby('asset_type').agg({
    'title': 'count',
    'confidence': 'mean'
})
print(type_stats)

🚀 PIPELINE COMPLET - NEWS + SENTIMENT + ASSET MAPPING

📰 RÉCUPÉRATION PAR CATÉGORIES (Top Headlines)

🔍 Catégorie: business...
   ✓ 30 articles

🔍 Catégorie: health...
   ✓ 55 articles

🔍 Catégorie: science...
   ✓ 55 articles

🔍 Catégorie: technology...
   ✓ 68 articles

🎯 RÉCUPÉRATION PAR ACTIFS (Mots-clés spécifiques)

🔍 Actif: SP500 (Index)...
   Query: "S&P 500" OR "SP500" OR "Wall Street" OR "Dow Jones" OR "Nasdaq" OR "Trump" OR "Fed" OR "Federal Res...
   ✓ 87 articles

🔍 Actif: CAC40 (Index)...
   Query: "CAC 40" OR "CAC40" OR "Paris Bourse" OR "Euronext Paris" OR "Macron" OR "French economy" OR "France...
   ✓ 93 articles

🔍 Actif: GER30 (Index)...
   Query: "DAX" OR "DAX 30" OR "GER30" OR "Frankfurt" OR "German economy" OR "Scholz" OR "Germany stocks"...
   ✓ 95 articles

🔍 Actif: AAPL (Company)...
   Query: "Apple" OR "AAPL" OR "iPhone" OR "iPad" OR "Mac" OR "Tim Cook" OR "iOS" OR "App Store"...
   ✓ 94 articles

🔍 Actif: AMZN (Company)...
   Query: "Amazon" OR "AMZN" OR "AW

In [ ]:
# Exemple 2 : News sur des entreprises spécifiques
df_tech = analyze_news_pipeline(
    query="Apple OR Tesla OR Microsoft OR Google",
    page_size=100
)

# Filtrer par confiance élevée (> 80%)
df_high_confidence = df_tech[df_tech['confidence'] > 0.8]

print(f"\n🎯 News avec confiance > 80% : {len(df_high_confidence)} articles")
print(df_high_confidence[['title', 'sentiment', 'confidence']])

🔄 DÉBUT DU PIPELINE

📡 Récupération des news (query='Apple OR Tesla OR Microsoft OR Google')...
✓ 0 articles récupérés
❌ Aucune news récupérée


KeyError: 'confidence'

In [ ]:
# Exemple 3 : Analyse par sentiment
print("\n📈 STATISTIQUES")
print(f"Sentiment positif : {(df_tech['sentiment'] == 'Positive').sum()} articles")
print(f"Sentiment négatif : {(df_tech['sentiment'] == 'Negative').sum()} articles")
print(f"\nConfiance moyenne : {df_tech['confidence'].mean():.2%}")


📈 STATISTIQUES
Sentiment positif : 52 articles
Sentiment négatif : 46 articles

Confiance moyenne : 87.96%


In [11]:
# Nouvelle cellule après les exemples

## BONUS: Comparaison avec/sans filtre

# Sans filtre
df_no_filter = analyze_news_pipeline(
    page_size=50,
    categories=['business'],
    use_asset_keywords=False,
    apply_filter=False  # Désactive le filtre
)

# Avec filtre
df_with_filter = analyze_news_pipeline(
    page_size=50,
    categories=['business'],
    use_asset_keywords=False,
    apply_filter=True  # Active le filtre
)

print("\n" + "="*80)
print("📊 COMPARAISON AVANT/APRÈS FILTRE")
print("="*80)
print(f"\nSans filtre: {len(df_no_filter)} articles")
print(f"Avec filtre: {len(df_with_filter)} articles")
print(f"Bruit éliminé: {len(df_no_filter) - len(df_with_filter)} ({(len(df_no_filter) - len(df_with_filter)) / len(df_no_filter) * 100:.1f}%)")

print("\n📰 Exemples d'articles ÉLIMINÉS par le filtre:")
removed = df_no_filter[~df_no_filter['url'].isin(df_with_filter['url'])]
print(removed[['title', 'source', 'asset_ticker']].head(10))

🚀 PIPELINE COMPLET - NEWS + SENTIMENT + ASSET MAPPING

📰 RÉCUPÉRATION PAR CATÉGORIES (Top Headlines)

🔍 Catégorie: business...
   ✓ 46 articles

✅ TOTAL: 46 articles uniques (sur 46 récupérés)


🤖 ANALYSE DE SENTIMENT + MAPPING ACTIFS
   ✓ Analysé 25/46 news...

✅ 67 associations news-actifs analysées

🚀 PIPELINE COMPLET - NEWS + SENTIMENT + ASSET MAPPING

📰 RÉCUPÉRATION PAR CATÉGORIES (Top Headlines)

🔍 Catégorie: business...
   ✓ 46 articles

✅ TOTAL: 46 articles uniques (sur 46 récupérés)


🤖 ANALYSE DE SENTIMENT + MAPPING ACTIFS
   ✓ Analysé 25/46 news...

🧹 FILTRAGE NON-FINANCIER

🔍 FILTRAGE NIVEAU 1 - Sources exclues
   Avant: 67 news
   Après: 64 news (-3)

🔍 FILTRAGE NIVEAU 2 - Mots-clés non-financiers
   Avant: 64 news
   Après: 58 news (-6)

✅ TOTAL FILTRÉ: 67 → 58 news
   Éliminées: 9 (13.4%)

✅ 58 associations news-actifs analysées


📊 COMPARAISON AVANT/APRÈS FILTRE

Sans filtre: 67 articles
Avec filtre: 58 articles
Bruit éliminé: 9 (13.4%)

📰 Exemples d'articles ÉLIMINÉS p

## 7. Export des résultats (pour intégration équipe)

In [11]:
# Sauvegarder en CSV
output_file = f"sentiment_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_finance.to_csv(output_file, index=False)
print(f"✓ Résultats sauvegardés dans : {output_file}")

# Ou en JSON pour API
output_json = output_file.replace('.csv', '.json')
df_finance.to_json(output_json, orient='records', indent=2)
print(f"✓ Résultats JSON dans : {output_json}")

✓ Résultats sauvegardés dans : sentiment_analysis_20260204_113632.csv
✓ Résultats JSON dans : sentiment_analysis_20260204_113632.json


## 8. Classification Financière des News (Gatekeeper DistilBERT)

### Chargement du modèle de classification binaire Financial/Non-Financial

In [13]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

class FinancialNewsClassifier:
    """Classification binaire: Financière vs Non-Financière"""
    
    def __init__(self, model_path, device='cpu'):
        self.device = device
        print(f"📦 Chargement du classificateur DistilBERT depuis {model_path}...")
        self.tokenizer = DistilBertTokenizer.from_pretrained(model_path)
        self.model = DistilBertForSequenceClassification.from_pretrained(model_path)
        self.model = self.model.to(device)
        self.model.eval()
        print("✓ Classificateur chargé et prêt")
    
    def predict(self, text, threshold=0.5):
        """
        Prédit si le texte est une news financière
        
        Args:
            text: Texte à classifier
            threshold: Seuil de décision (défaut: 0.5)
        
        Returns:
            dict avec is_financial (bool), confidence (float), label (str)
        """
        if not text or len(text.strip()) == 0:
            return {
                'is_financial': 0,
                'confidence': 0.5,
                'label': 'Unknown'
            }
        
        # Tokenization
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        
        # Prédiction
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
            probabilities = torch.softmax(outputs.logits, dim=1)
            financial_prob = probabilities[0][1].item()
        
        is_financial = int(financial_prob >= threshold)
        
        return {
            'is_financial': is_financial,
            'confidence': financial_prob,
            'label': 'Financial' if is_financial else 'Non-Financial'
        }
    
    def batch_predict(self, texts, threshold=0.5, batch_size=16):
        """Prédiction par batch pour optimiser les performances"""
        results = []
        
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            
            # Tokenization du batch
            encodings = self.tokenizer(
                batch_texts,
                add_special_tokens=True,
                max_length=256,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
            
            input_ids = encodings['input_ids'].to(self.device)
            attention_mask = encodings['attention_mask'].to(self.device)
            
            # Prédiction
            with torch.no_grad():
                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                probabilities = torch.softmax(outputs.logits, dim=1)
                financial_probs = probabilities[:, 1].cpu().numpy()
            
            # Conversion en résultats
            for prob in financial_probs:
                is_financial = int(prob >= threshold)
                results.append({
                    'is_financial': is_financial,
                    'confidence': float(prob),
                    'label': 'Financial' if is_financial else 'Non-Financial'
                })
        
        return results

# Charger le classificateur
CLASSIFIER_MODEL_PATH = "./financial_news_classifier/best_model"

financial_classifier = FinancialNewsClassifier(
    CLASSIFIER_MODEL_PATH, 
    device=device
)

print("\n✅ Classificateur DistilBERT prêt pour filtrage Financial/Non-Financial")

📦 Chargement du classificateur DistilBERT depuis ./financial_news_classifier/best_model...
✓ Classificateur chargé et prêt

✅ Classificateur DistilBERT prêt pour filtrage Financial/Non-Financial


### Test du classificateur sur quelques exemples

In [14]:
## Test du classificateur

test_texts = [
    "Apple stock hits record high on strong iPhone sales",
    "Best video games to play this summer",
    "Federal Reserve raises interest rates to combat inflation",
    "Celebrity gossip: Hollywood stars reunite",
    "Tesla announces new Gigafactory in Texas",
    "Top 10 workout tips for beginners"
]

print("="*80)
print("🧪 TEST DU CLASSIFICATEUR FINANCIER")
print("="*80)

for text in test_texts:
    result = financial_classifier.predict(text)
    emoji = "💰" if result['is_financial'] else "🚫"
    print(f"\n{emoji} [{result['label']}] (confiance: {result['confidence']:.2%})")
    print(f"   '{text}'")

🧪 TEST DU CLASSIFICATEUR FINANCIER

💰 [Financial] (confiance: 99.54%)
   'Apple stock hits record high on strong iPhone sales'

🚫 [Non-Financial] (confiance: 0.05%)
   'Best video games to play this summer'

💰 [Financial] (confiance: 99.95%)
   'Federal Reserve raises interest rates to combat inflation'

🚫 [Non-Financial] (confiance: 0.87%)
   'Celebrity gossip: Hollywood stars reunite'

🚫 [Non-Financial] (confiance: 0.68%)
   'Tesla announces new Gigafactory in Texas'

🚫 [Non-Financial] (confiance: 0.06%)
   'Top 10 workout tips for beginners'


### Application du classificateur sur le CSV existant

In [15]:
## Charger le CSV existant (dernier fichier généré)
import glob
import os

# Trouver le dernier CSV de sentiment
csv_files = glob.glob("sentiment_analysis_*.csv")
if csv_files:
    latest_csv = max(csv_files, key=os.path.getctime)
    print(f"📂 Chargement du CSV: {latest_csv}")
    df_existing = pd.read_csv(latest_csv)
    print(f"   {len(df_existing)} news chargées")
else:
    print("❌ Aucun CSV trouvé. Utilisez 'df_finance' du notebook.")
    df_existing = df_finance.copy()

print(f"\n📊 Aperçu des colonnes:")
print(df_existing.columns.tolist())

📂 Chargement du CSV: sentiment_analysis_20260204_113632.csv
   975 news chargées

📊 Aperçu des colonnes:
['asset_ticker', 'asset_type', 'query_keyword', 'news_category', 'title', 'description', 'source', 'published_at', 'sentiment', 'confidence', 'prob_negative', 'prob_positive', 'url']


In [16]:
## Application du classificateur sur toutes les news

print("\n" + "="*80)
print("🤖 CLASSIFICATION FINANCIÈRE DE TOUTES LES NEWS")
print("="*80)

# Créer le texte complet (titre + description)
df_existing['full_text'] = (
    df_existing['title'].fillna('') + ' ' + 
    df_existing['description'].fillna('')
)

# Classification par batch (plus rapide)
print(f"\n🔄 Classification de {len(df_existing)} news...")
texts_to_classify = df_existing['full_text'].tolist()

classification_results = financial_classifier.batch_predict(
    texts_to_classify, 
    threshold=0.5,
    batch_size=32
)

# Ajouter les résultats au DataFrame
df_existing['is_financial'] = [r['is_financial'] for r in classification_results]
df_existing['financial_confidence'] = [r['confidence'] for r in classification_results]
df_existing['financial_label'] = [r['label'] for r in classification_results]

# Supprimer la colonne temporaire
df_existing = df_existing.drop('full_text', axis=1)

print(f"✅ Classification terminée!")

# Statistiques
print(f"\n📊 RÉSULTATS DE CLASSIFICATION:")
print(f"   News financières (is_financial=1): {(df_existing['is_financial'] == 1).sum()} ({(df_existing['is_financial'] == 1).sum() / len(df_existing) * 100:.1f}%)")
print(f"   News non-financières (is_financial=0): {(df_existing['is_financial'] == 0).sum()} ({(df_existing['is_financial'] == 0).sum() / len(df_existing) * 100:.1f}%)")
print(f"   Confiance moyenne: {df_existing['financial_confidence'].mean():.2%}")

# Aperçu
print(f"\n📰 APERÇU DES RÉSULTATS:")
print(df_existing[['title', 'is_financial', 'financial_label', 'financial_confidence', 'sentiment']].head(15))


🤖 CLASSIFICATION FINANCIÈRE DE TOUTES LES NEWS

🔄 Classification de 975 news...
✅ Classification terminée!

📊 RÉSULTATS DE CLASSIFICATION:
   News financières (is_financial=1): 254 (26.1%)
   News non-financières (is_financial=0): 721 (73.9%)
   Confiance moyenne: 26.46%

📰 APERÇU DES RÉSULTATS:
                                                title  is_financial  \
0   S&P 500 futures climb after index starts Febru...             1   
1   S&P 500 futures climb after index starts Febru...             1   
2   China Bans Hidden Car Door Handles in World-Fi...             0   
3   Switch has now outsold DS, becomes best-sellin...             0   
4   Switch has now outsold DS, becomes best-sellin...             0   
5   Powerball: See the winning numbers in Monday’s...             1   
6   Asian Stocks Set to Climb After US Data Lifts ...             1   
7   Elon Musk's SpaceX confirms it is taking over ...             0   
8   Elon Musk's SpaceX confirms it is taking over ...          

### Analyse des résultats par catégorie

In [17]:
## Analyse détaillée par actif et catégorie

print("="*80)
print("📊 ANALYSE PAR ACTIF")
print("="*80)

asset_analysis = df_existing.groupby('asset_ticker').agg({
    'title': 'count',
    'is_financial': 'sum',
    'financial_confidence': 'mean'
}).rename(columns={
    'title': 'total_news',
    'is_financial': 'financial_news',
    'financial_confidence': 'avg_confidence'
})
asset_analysis['financial_ratio'] = (asset_analysis['financial_news'] / asset_analysis['total_news'] * 100).round(1)
asset_analysis = asset_analysis.sort_values('total_news', ascending=False)

print(asset_analysis.head(15))

print("\n" + "="*80)
print("📊 ANALYSE PAR TYPE D'ACTIF")
print("="*80)

type_analysis = df_existing.groupby('asset_type').agg({
    'title': 'count',
    'is_financial': 'sum',
    'financial_confidence': 'mean'
}).rename(columns={
    'title': 'total_news',
    'is_financial': 'financial_news',
    'financial_confidence': 'avg_confidence'
})
type_analysis['financial_ratio'] = (type_analysis['financial_news'] / type_analysis['total_news'] * 100).round(1)

print(type_analysis)

# Exemples de news NON-financières détectées
print("\n" + "="*80)
print("🚫 EXEMPLES DE NEWS NON-FINANCIÈRES DÉTECTÉES")
print("="*80)

non_financial = df_existing[df_existing['is_financial'] == 0].sort_values('financial_confidence')
print(non_financial[['title', 'source', 'financial_label', 'financial_confidence']].head(10))

📊 ANALYSE PAR ACTIF
              total_news  financial_news  avg_confidence  financial_ratio
asset_ticker                                                             
TSLA                 130              20        0.166121             15.4
HO                   124              11        0.090774              8.9
AAPL                  80              10        0.134594             12.5
CAC40                 77              21        0.277580             27.3
SP500                 75              35        0.466191             46.7
OIL                   64              48        0.743126             75.0
GER30                 63              14        0.223973             22.2
AMZN                  60              18        0.305016             30.0
MC                    59              16        0.263810             27.1
ENGI                  48              11        0.249496             22.9
GOLD                  48              20        0.417234             41.7
SAN               

### Export du CSV avec classification financière

In [18]:
## Sauvegarder le nouveau CSV avec classification financière

output_filename = f"hybrid_news_financial_classified_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

df_existing.to_csv(output_filename, index=False)

print("="*80)
print("💾 EXPORT TERMINÉ")
print("="*80)
print(f"\n✅ CSV sauvegardé: {output_filename}")
print(f"   Total: {len(df_existing)} news")
print(f"   Colonnes: {len(df_existing.columns)}")
print(f"\n📋 Colonnes incluses:")
for col in df_existing.columns:
    print(f"   • {col}")

# Export JSON également
output_json = output_filename.replace('.csv', '.json')
df_existing.to_json(output_json, orient='records', indent=2)
print(f"\n✅ JSON sauvegardé: {output_json}")

💾 EXPORT TERMINÉ

✅ CSV sauvegardé: hybrid_news_financial_classified_20260204_114901.csv
   Total: 975 news
   Colonnes: 16

📋 Colonnes incluses:
   • asset_ticker
   • asset_type
   • query_keyword
   • news_category
   • title
   • description
   • source
   • published_at
   • sentiment
   • confidence
   • prob_negative
   • prob_positive
   • url
   • is_financial
   • financial_confidence
   • financial_label

✅ JSON sauvegardé: hybrid_news_financial_classified_20260204_114901.json


### 📌 Pipeline complet recommandé : Filtrage en 2 étapes

**ÉTAPE 1** : Classification binaire DistilBERT (is_financial)  
**ÉTAPE 2** : Analyse sentiment FinBERT (uniquement si is_financial=1)

Ceci permet d'économiser des ressources en évitant d'analyser le sentiment de news non-financières.

In [ ]:
## BONUS : Filtrer uniquement les news financières pour analyse approfondie

df_financial_only = df_existing[df_existing['is_financial'] == 1].copy()

print("="*80)
print("💰 DATASET FILTRÉ - NEWS FINANCIÈRES UNIQUEMENT")
print("="*80)
print(f"\nAvant filtrage: {len(df_existing)} news")
print(f"Après filtrage: {len(df_financial_only)} news financières")
print(f"Éliminées: {len(df_existing) - len(df_financial_only)} news non-financières ({(len(df_existing) - len(df_financial_only)) / len(df_existing) * 100:.1f}%)")

# Statistiques sentiment sur news financières uniquement
print(f"\n📊 SENTIMENT DES NEWS FINANCIÈRES:")
print(f"   Positive: {(df_financial_only['sentiment'] == 'Positive').sum()} ({(df_financial_only['sentiment'] == 'Positive').sum() / len(df_financial_only) * 100:.1f}%)")
print(f"   Negative: {(df_financial_only['sentiment'] == 'Negative').sum()} ({(df_financial_only['sentiment'] == 'Negative').sum() / len(df_financial_only) * 100:.1f}%)")
print(f"   Confiance sentiment moyenne: {df_financial_only['confidence'].mean():.2%}")
print(f"   Confiance classification moyenne: {df_financial_only['financial_confidence'].mean():.2%}")

# Export du dataset filtré
output_filtered = f"financial_news_only_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_financial_only.to_csv(output_filtered, index=False)
print(f"\n✅ Dataset filtré sauvegardé: {output_filtered}")